# Computation of annotation enrichment in a genetic set 

This code excerpt shows how to use the annotation enrichment computation function in a genetic set.
The funciton return a dataframe of binom and hypergeometric p-value (<= 0.05) and for each p-value a fdr correction with alpha = 0.05.   

In the second part of the code we will compare the results obtained with the Great webserver and between the binomial and hypergeometric probability

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import greatpy as grt
import pandas as pd
from math import inf
from numpy import log,nan, int64,cov,corrcoef
from scipy.stats import pearsonr
from seaborn import scatterplot as sp

import warnings
warnings.filterwarnings('ignore')

import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects import r as r
pandas2ri.activate()
import pandas as pd

import os 
import matplotlib.pyplot as plt
import re
import time 

In [3]:
import pandas as pd
from math import lgamma, log, exp,fabs,inf
from scipy.special import comb
from statsmodels.stats.multitest import multipletests
import dask.dataframe as dd 
import numpy as np 

pd.options.display.float_format = '{:12.5e}'.format

def get_association(test,regdom): 
    """
    Function allowing from a file of genomic regions from CHIPseq 
    and a file of genomic regulatory domains to determine the names 
    of genes associated with at least one genomic region 

    Parameters
    ----------
    test : pd.dataFrame
        df of the tests pics => columns: ["Chr","Chr_Start","Chr_End"]
    
    regdom : pd.dataFrame
        df of the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

    Returns
    -------
    res : list
        list of gene associated with at least with one test peak
        
    Exemples 
    --------
    test = pd.DataFrame({
    ...    "Chr":["chr1"],
    ...    "Chr_Start":[1052028],
    ...    "Chr_End": [1052049]})

    regdom = pd.DataFrame({
    ...    "Chr":["chr1","chr1"],
    ...    "Chr_Start":[1034992,1079306],
    ...    "Chr_End": [1115089,1132016],
    ...    "Name":["RNF223","C1orf159"],
    ...    "tss":[1074306,1116089],
    ...    "strand":['-','-']})

    >>> get_association(test,regdom)        
        ['RNF223']
    
    """
    res = []
    for i in range(test.shape[0]) :
        currTest = test.iloc[i]
        regdom_curr_test = regdom.loc[(regdom["Chr"] == currTest["Chr"])].sort_values("Chr_Start")
        regdom_curr_test = regdom_curr_test.loc[
            ((regdom_curr_test["Chr_Start"] <= currTest["Chr_Start"]) & (regdom_curr_test["Chr_End"] >= currTest["Chr_End"])) | # regdom overlap totally test 
            ((regdom_curr_test["Chr_Start"] >= currTest["Chr_Start"]) & (regdom_curr_test["Chr_End"] <= currTest["Chr_End"])) | # test overlap totally regdom 
            ((regdom_curr_test["Chr_Start"] <= currTest["Chr_Start"]) & (regdom_curr_test["Chr_End"] <= currTest["Chr_End"]) & (regdom_curr_test["Chr_End"] >= currTest["Chr_Start"])) | # regdom overlap not totally test on left side 
            ((regdom_curr_test["Chr_Start"] >= currTest["Chr_Start"]) & (regdom_curr_test["Chr_End"] >= currTest["Chr_End"]) & (regdom_curr_test["Chr_Start"] <= currTest["Chr_End"])) # regdom overlap not totally test on right side 
            ] 
        res = res + list(regdom_curr_test["Name"])
    return list(dict.fromkeys(res))

def len_regdom(regdom:pd.DataFrame): 
    """
    Function to calculate for each gene name from regdom the
     size of the regulatory region for this gene in the genome 

    Parameters
    ----------    
    regdom : pd.dataFrame
        df of the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

    Returns
    -------
    dict
        dictionary in which each key corresponds to a gene name 
        from regdom and the value is the size of the regulatory 
        region for that gene
        
    Exemples 
    --------
    regdom = pd.DataFrame({
    ...    "Chr":["chr1","chr1"],
    ...    "Chr_Start":[1034992,1079306],
    ...    "Chr_End": [1115089,1132016],
    ...    "Name":["RNF223","C1orf159"],
    ...    "tss":[1074306,1116089],
    ...    "strand":['-','-']}))

    >>> len_regdom(regdom)
        {'RNF223': 80097, 'C1orf159': 52710}

    """
    test = regdom["Chr_End"]-regdom["Chr_Start"]
    return pd.DataFrame({"len":list(test)},index = regdom["Name"]).to_dict()["len"]

def number_of_hit(test,regdom): 
    """ 
    Function to calculate the number of hits from several 
    genomic regions and the file describing the regulatory regions

    Parameters
    ----------
    test : pd.dataFrame
        df of the tests pics => columns: ["Chr","Chr_Start","Chr_End"]
    
    regdom : pd.dataFrame
        df of the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

    Returns
    -------
    nb : int
        number of hit 
        
    Exemples 
    --------
    test = pd.DataFrame({
    ...    "Chr":["chr1"],
    ...    "Chr_Start":[1052028],
    ...    "Chr_End": [1052049]})

    regdom = pd.DataFrame({
    ...    "Chr":["chr1","chr1"],
    ...    "Chr_Start":[1034992,1079306],
    ...    "Chr_End": [1115089,1132016],
    ...    "Name":["RNF223","C1orf159"],
    ...    "tss":[1074306,1116089],
    ...    "strand":['-','-']})

    >>> number_of_hit(test,regdom)        
        1
    
    """
    nb = 0
    regdom = regdom[["Chr","Chr_Start","Chr_End"]]
    for i in range(test.shape[0]) : 
        chrom = test.iat[i,0]
        start = test.iat[i,1]
        end = test.iat[i,2]
        regdom_np = regdom["Chr"].to_numpy()
        reg_start = regdom["Chr_Start"].to_numpy()
        reg_end = regdom["Chr_End"].to_numpy()
        Chr_reduce = np.where(regdom_np == chrom)
        reg_start = np.take(reg_start,Chr_reduce,axis = 0)[0]
        reg_end = np.take(reg_end,Chr_reduce,axis = 0)[0]

        if any((reg_start <= start) & (reg_end >= end)):  
            nb += 1
    return nb

def betacf(a,b,x): 
    """ Used by betai: Evaluates continued fraction for incomplete beta function """
    maxit = 10000
    eps = 3.0e-7 
    fpmin = 1.0e-30
    qab = a+b
    qap = a+1
    qam = a-1
    c = 1
    d = 1-qab*x/qap
    if fabs(d) < fpmin :
        d = fpmin
    d = 1/d 
    h = d
    for m in range(1,maxit+1) : 
        m2 = 2*m
        aa = m*(b-m)*x / ((qam+m2) * (a+m2))
        d = 1.0+aa*d
        if (fabs(d) < fpmin) : 
            d = fpmin
        c = 1.0+aa/c
        if (fabs(c) < fpmin):
            c = fpmin
        d = 1.0/d
        h *= d*c
        aa = -(a+m) * (qab+m)*x / ((a+m2) * (qap+m2))
        d = 1.0+aa*d  
        if (fabs(d) < fpmin):
            d = fpmin
        c = 1.0+aa/c
        if (fabs(c) < fpmin):
            c = fpmin
        d = 1.0/d
        dell = d*c
        h *= dell
        if (fabs(dell-1.0) < eps):
            break
    if (m > maxit):
        print("a or b too big, or MAXIT too small in betacf")
        return False
    return h

def betai(a,b,x):
    """Returns the incomplete beta function Ix(a, b)."""
    if x < 0 or x > 1 : 
        # print("bad x in routine betai")
        return False
    if x == 0 or x == 1 : 
        bt = 0.0
    else : 
        bt = exp(lgamma(a+b)-lgamma(a)-lgamma(b)+a*log(x)+b*log(1.0-x))
    if x < (a+1) / (a+b+2) : 
        return bt * betacf(a,b,x)/a
    return 1 - bt*betacf(b,a,1-x)/b

def get_binom_pval(n:int,k:int,p:float) -> float:
    """
    This function allows to calculate the binomial probability 
    of obtaining k in a set of size n and whose probability is p 

    Parameters
    ----------
    n : int
        Number of genomic region in the test set 
    k : int 
        Number of test genomic regions in the regulatory domain of a gene with annotation
    p : float
        Percentage of genome annotated

    Returns
    -------
    float
        binomial probability
        
    Exemples 
    --------
    >>> get_binom_pval(100,2,0.2)
        0.9999999947037065
    
    """
    if k == 0 : return 1
    else : return betai(k,n-k+1,p)

def hypergeom_pmf(N, K, n, k):
    """
    Function to calculate the probability mass function for hypergeometric distribution

    Parameters
    ----------
    N : int
        Total number of gene in the genome
    K : int 
        Number of genes in the genome with annotation
    n : int
        Number of gene in the test set
    k : int
        Number of genes in the test gene set with annotation

    Returns
    -------
    float
        proability mass function
        
    Exemples 
    --------
    >>> hypergeom_pmf(100,10,30,1)
        0.11270773995748315
    
    """
    Achoosex = comb(K,k,exact=True) 
    NAchoosenx = comb(N-K, n-k,exact=True) 
    Nchoosen = comb(N,n,exact=True) 
    return ((Achoosex)*NAchoosenx)/Nchoosen 

def hypergeom_cdf(N, K, n, k):
    """
    Function to calculate the cumulative density funtion for hypergeometric distribution

    Parameters
    ----------
    N : int
        Total number of gene in the genome
    K : int 
        Number of genes in the genome with annotation
    n : int
        Number of gene in the test set
    k : int
        Number of genes in the test gene set with annotation

    Returns
    -------
    float
        Cumulative density function
        
    Exemples 
    --------
    >>> hypergeom_cdf(100,10,30,1)
        0.9770827595419788
    
    """
    return np.sum([hypergeom_pmf(N, K, n, x) for x in range(k,min(K,n)+1)])

class GREAT: 
    def loader(test_data:None or str or pd.DataFrame,regdom_file:None or str or pd.DataFrame,chr_size_file:None or str or pd.DataFrame,annotation_file:None or str or pd.DataFrame):
        """
        This function is used to load all datasets needed for the enrichment calculation

        Parameters
        ----------
        test_data : str or pd.DataFrame
            Genomic set of peaks to be tested
        regdom_file : str or pd.DataFrame 
            Regulatory domain of all genes in the genome 
        chr_size_file : str or pd.DataFrame
            Table with the size of each chromosome
        annotation_file : str or pd.DataFrame
            Table with the annotation of each gene in the genome

        Returns
        -------
        test_data : pd.DataFrame
            Genomic set of peaks to be tested in the good format 
        regdom : pd.DataFrame
            Regulatory domain of all genes in the genome in the good format
        size : pd.DataFrame
            Table with the size of each chromosome in the good format
        ann : pd.DataFrame
            Table with the annotation of each gene in the genome in the good format
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader(
        ...    "../../data/tests/test_data/input/02_srf_hg38.bed",
        ...    "../../data/human/hg38/regulatory_domain.bed",
        ...    "../../data/human/hg38/chr_size.bed",
        ...    "../../data/human/ontologies.csv"
        ...    )

        >>> test.head()
            |    | Chr   |   Chr_Start |   Chr_End |
            |---:|:------|------------:|----------:|
            |  0 | chr1  |     1052028 |   1052049 |
            |  1 | chr1  |     1065512 |   1065533 |
            |  2 | chr1  |     1067375 |   1067397 |
            |  3 | chr1  |     1068083 |   1068119 |
            |  4 | chr1  |    10520283 |  10520490 |

        >>> regdom.head()
            |    | Chr   |   Chr_Start |   Chr_End | Name      |   tss | Strand   |
            |---:|:------|------------:|----------:|:----------|------:|:---------|
            |  0 | chr1  |           0 |     22436 | MIR6859-1 | 17436 | -        |
            |  1 | chr1  |       16436 |     22436 | MIR6859-2 | 17436 | -        |
            |  2 | chr1  |       16436 |     22436 | MIR6859-3 | 17436 | -        |
            |  3 | chr1  |       16436 |     28370 | MIR6859-4 | 17436 | -        |
            |  4 | chr1  |       22436 |     34370 | WASH7P    | 29370 | -        |

        >>> size.head()
            |    | Chrom   |      Size |
            |---:|:--------|----------:|
            |  0 | chr1    | 248956422 |
            |  1 | chr2    | 242193529 |
            |  2 | chr3    | 198295559 |
            |  3 | chr4    | 190214555 |
            |  4 | chr5    | 181538259 |
            
        >>> ann.head()
            |    | id         | name                                                   | symbol        |
            |---:|:-----------|:-------------------------------------------------------|:--------------|
            |  0 | GO:0003924 | GTPase activity                                        | DNAJC25-GNG10 |
            |  1 | GO:0007186 | G protein-coupled receptor signaling pathway           | DNAJC25-GNG10 |
            |  2 | GO:0003723 | RNA binding                                            | NUDT4B        |
            |  3 | GO:0005829 | cytosol                                                | NUDT4B        |
            |  4 | GO:0008486 | diphosphoinositol-polyphosphate diphosphatase activity | NUDT4B        |
        
        """

        if type(regdom_file) == str :
            regdom = pd.read_csv(regdom_file,sep = "\t",comment="#",
                        names = ["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],
                        dtype = {"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"object","tss":"int64","Strand":"object"})
        elif type(regdom_file) == pd.DataFrame :
            regdom = regdom_file.iloc[:,:6]
            colname = list(regdom.columns)
            try : 
                regdom = regdom.rename(columns = {colname[0]:"Chr",colname[1]:"Chr_Start",colname[2]:"Chr_End",colname[3]:"Name",colname[4]:"tss",colname[5]:"Strand"})
            except :
                print("Error in the format of the regdom file")
                print("The regdom file must have the following columns : Chr, Chr_Start, Chr_End, Name, tss, Strand")
                return False 
        else : 
            regdom = regdom_file

        if type(test_data) == str : 
            test_data = pd.read_csv(test_data,sep = "\t",comment = "#",usecols = [0,1,2],
                            names = ["Chr", "Chr_Start", "Chr_End"],
                            dtype = {"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64"})
        elif type(test_data) == pd.DataFrame : 
            test_data = test_data.iloc[:,:3]
            colname = list(test_data.columns)
            try : 
                test_data = test_data.rename(columns={colname[0]:"Chr",colname[1]:"Chr_Start",colname[2]:"Chr_End"})
            except : 
                print("Error in test dataframe, please check your input")
                print("Columns should be : chr...(type object), start(type int), end(type int)")
                return False
        else :
            pass

        if type(chr_size_file) == str :
            size = pd.read_csv(chr_size_file,sep = "\t",comment = "#",
                            names = ["Chrom","Size"],
                            dtype = {"Chrom":"object", "Size":"int64"})
        elif type(chr_size_file) == pd.DataFrame :
            size = chr_size_file.iloc[:,:2]
            colname = list(size.columns)
            try : 
                size = size.rename(columns = {colname[0]:"Chrom",colname[1]:"Size"})
            except : 
                print("Error in the format of the chr_size file")
                print("The chr_size file must have the following columns : Chrom, Size")
                return False
        else :
            size = chr_size_file

        if type(annotation_file) == str : 
            dask_df = dd.read_csv(annotation_file,sep = ";",  comment = "#",
                            dtype = {"ensembl":"object","id":"object","name":"object","ontology.group":"object","gene.name":"object","symbol":"object"},
                            usecols = ["id","name","symbol"],low_memory = False)
            ann = dask_df.compute()
            ann = ann[ann['id'].str.match('^GO.*')== True]
        elif type(annotation_file) == pd.DataFrame : 
            ann = annotation_file.iloc[:,:4]
            colname = list(ann.columns)
            try : 
                ann = ann.rename(columns={colname[0]:"id",colname[1]:"name",colname[3]:"symbol"})
            except : 
                print("Error in the format of the annotation file")
                print("The annotation file must have the following columns : id, name, symbol")
                return False
        else :
            ann = annotation_file

        return test_data,regdom,size,ann

    def __enrichment_binom_and_hypergeom(test,regdom,size,ann,asso) : 
        """
        This private function is used to compute the enrichment of the test data using the binomial test and the hypergeometric test.

        Parameters
        ----------
        test : pd.DataFrame
            Genomic set of peaks to be tested
        regdom : pd.DataFrame 
            Regulatory domain of all genes in the genome 
        chr_size :  pd.DataFrame
            Table with the size of each chromosome
        annotation : pd.DataFrame
            Table with the annotation of each gene in the genome
        asso : list 
            List of the association between gene from regdom and peaks from test

        Returns
        -------
        pd.DataFrame
            dataframe contains for every GO ID associate with a every associated gene the p-value for the binomial test and the the hypergeometric test
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader("../../data/tests/test_data/input/02_srf_hg38.bed","../../data/human/hg38/regulatory_domain.bed","../../data/human/hg38/chr_size.bed","../../data/human/ontologies.csv")
        >>> enrichment = GREAT.____enrichment_binom_and_hypergeom(
        ...    test = test,
        ...    regdom = regdom,
        ...    size = size,
        ...    ann = ann,
        ...    asso = get_association(test,regdom)
        ...    )

        >>> enrichment.head()
            |            | go_term                                                          |   binom_p_value |   hypergeom_p_value |
            |:-----------|:-----------------------------------------------------------------|----------------:|--------------------:|
            | GO:0045887 | positive regulation of synaptic growth at neuromuscular junction |     5.17744e-13 |          0.0029275  |
            | GO:0044721 | protein import into peroxisome matrix, substrate release         |     4.83812e-10 |          0.0029275  |
            | GO:0036250 | peroxisome transport along microtubule                           |     4.83812e-10 |          0.0029275  |
            | GO:0016561 | protein import into peroxisome matrix, translocation             |     6.31131e-10 |          0.00584656 |
            | GO:0047485 | protein N-terminus binding                                       |     1.2945e-09  |          0.0050377  |
        """
        # Init Great
        res = {}
        hit = {}

        # init Hypergeom
        hypergeom_gene_set = len(asso) # get the number of genes in the test gene set.
        hypergeom_total_number_gene = regdom.shape[0] #get the number of genes in the genome.

        # Init binom 
        n_binom = test.shape[0]# get the number of genomic region in the test set
        total_nu = size["Size"].sum()# get the total number of nucleotides in the genome

        ann_red = ann[ann["symbol"].isin(asso)]
        regdom = regdom[regdom["Name"].isin(list(ann[ann["id"].isin(list(ann_red["id"]))]["symbol"]))]#reduction of the regdom file by selecting only the genes whose GO ID is owned by a gene of the association 
        len_on_chr = len_regdom(regdom)# get the length of each regulatory domain 

        #Compute for all associating gene and for each GO id associated with the gene the probability. 
        for name in asso :
            ann_name_gene = ann[ann["symbol"].isin([name])]
            id = ann_name_gene["id"]
            tmp = []
            for i in (list(id.unique())) : 
                gene_imply = ann[ann['id'].isin([i])]
                K_hypergeom = gene_imply.shape[0] # get be the number of genes in the genome with annotation
                curr_regdom = regdom.loc[regdom["Name"].isin(list(gene_imply["symbol"]))]
                k_hypergeom = curr_regdom.loc[curr_regdom["Name"].isin(asso)].shape[0] # get the number of genes in the test gene set with annotation

                if i not in list(hit.keys()) : 
                    hit[i] = number_of_hit(test,curr_regdom)# get the number of test genomic regions in the regulatory domain of a gene with annotation
                k_binom = hit[i]
                nb_binom = sum([len_on_chr[i] for i in curr_regdom["Name"]])# get the portion of the genome in the regulatory domain of a gene with annotation
                tmp.append((k_binom,nb_binom,i,gene_imply.iloc[0]["name"],K_hypergeom,k_hypergeom))
            res.update({elem[2]:[ elem[3],get_binom_pval(n_binom,elem[0],elem[1]/total_nu),elem[0]/(elem[1]/total_nu),n_binom,elem[0],elem[1]/total_nu, hypergeom_cdf(hypergeom_total_number_gene,elem[4],hypergeom_gene_set,elem[5]),(elem[5]*hypergeom_total_number_gene)/(hypergeom_gene_set*elem[4]),hypergeom_total_number_gene,elem[4],hypergeom_gene_set,elem[5] ] for elem in tmp})
        return pd.DataFrame(res).transpose().rename(columns = {0:"go_term",1:"binom_p_value",2:"binom_fold_enrichment",3:"n_binom",4:"k_binom",5:"p_binom",6:"hypergeom_p_value",7:"hypergeometric_fold_enrichment",8:"N_hyper",9:"K_hyper",10:"n_hyper",11:"k_hyper"}).replace(0,np.nan).dropna().sort_values(by = "binom_p_value")
    
    def __enrichment_binom(test,regdom,size,ann,asso):
        """
        This private function is used to compute the enrichment of the test data using the binomial test.

        Parameters
        ----------
        test : pd.DataFrame
            Genomic set of peaks to be tested
        regdom : pd.DataFrame 
            Regulatory domain of all genes in the genome 
        chr_size :  pd.DataFrame
            Table with the size of each chromosome
        annotation : pd.DataFrame
            Table with the annotation of each gene in the genome
        asso : list 
            List of the association between gene from regdom and peaks from test

        Returns
        -------
        pd.DataFrame
            dataframe contains for every GO ID associate with a every associated gene the p-value for the binomial test
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader("../../data/tests/test_data/input/02_srf_hg38.bed","../../data/human/hg38/regulatory_domain.bed","../../data/human/hg38/chr_size.bed","../../data/human/ontologies.csv")
        >>> enrichment = GREAT.____enrichment_binom(
        ...    test = test,
        ...    regdom = regdom,
        ...    size = size,
        ...    ann = ann,
        ...    asso = get_association(test,regdom)
        ...    )

        >>> enrichment.head()
            |            | go_term                                                          |   binom_p_value |
            |:-----------|:-----------------------------------------------------------------|----------------:|
            | GO:0045887 | positive regulation of synaptic growth at neuromuscular junction |     5.17744e-13 |
            | GO:0044721 | protein import into peroxisome matrix, substrate release         |     4.83812e-10 |
            | GO:0036250 | peroxisome transport along microtubule                           |     4.83812e-10 |
            | GO:0016561 | protein import into peroxisome matrix, translocation             |     6.31131e-10 |
            | GO:0047485 | protein N-terminus binding                                       |     1.2945e-09  |
        """ 
        # Init Great
        res = {}
        hit = {}

        # Init binom 
        n_binom = test.shape[0]# get the number of genomic region in the test set
        total_nu = size["Size"].sum()# get the total number of nucleotides in the genome
        
        ann_red = ann[ann["symbol"].isin(asso)]
        regdom = regdom[regdom["Name"].isin(list(ann[ann["id"].isin(list(ann_red["id"]))]["symbol"]))]#reduction of the regdom file by selecting only the genes whose GO ID is owned by a gene of the association 
        len_on_chr = len_regdom(regdom)# get the length of each regulatory domain 

        #Compute for all associating gene and for each GO id associated with the gene the probability. 
        for name in asso :
            ann_name_gene = ann[ann["symbol"].isin([name])]
            id = ann_name_gene["id"]
            tmp = []
            for i in (list(id.unique())) : 
                gene_imply = ann[ann['id'].isin([i])]
                curr_regdom = regdom.loc[regdom["Name"].isin(list(gene_imply["symbol"]))]

                if i not in list(hit.keys()) : 
                    hit[i] = number_of_hit(test,curr_regdom)# get the number of test genomic regions in the regulatory domain of a gene with annotation
                k_binom = hit[i]
                nb_binom = sum([len_on_chr[i] for i in curr_regdom["Name"]])# get the portion of the genome in the regulatory domain of a gene with annotation
                tmp.append((k_binom,nb_binom,i,gene_imply.iloc[0]["name"]))
            res.update({elem[2]:[ elem[3],get_binom_pval(n_binom,elem[0],elem[1]/total_nu),elem[0]/(elem[1]/total_nu) ] for elem in tmp})
        return pd.DataFrame(res).transpose().rename(columns = {0:"go_term",1:"binom_p_value",2:"binom_fold_enrichment"}).sort_values(by = "binom_p_value")

    def __enrichment_hypergeom(test,regdom,ann,asso): 
        """
        This private function is used to compute the enrichment of the test data using the hypergeometric test.

        Parameters
        ----------
        test : pd.DataFrame
            Genomic set of peaks to be tested
        regdom : pd.DataFrame 
            Regulatory domain of all genes in the genome 
        chr_size :  pd.DataFrame
            Table with the size of each chromosome
        annotation : pd.DataFrame
            Table with the annotation of each gene in the genome
        asso : list 
            List of the association between gene from regdom and peaks from test

        Returns
        -------
        pd.DataFrame
            dataframe contains for every GO ID associate with a every associated gene the p-value for the hypergeometric test
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv")
        >>> enrichment = GREAT.____enrichment_hypergeom(
        ...    test = test,
        ...    regdom = regdom,
        ...    ann = ann,
        ...    asso = get_association(test,regdom)
        ...    )

        >>> enrichment.head()   
        """ 
        # Init Great
        res = {}

        # Init hypergeom
        hypergeom_total_number_gene = regdom.shape[0] #get the number of genes in the genome
        hypergeom_gene_set = len(asso) # get the number of genes in the test gene set.

        ann_red = ann[ann["symbol"].isin(asso)]
        regdom = regdom[regdom["Name"].isin(list(ann[ann["id"].isin(list(ann_red["id"]))]["symbol"]))]#reduction of the regdom file by selecting only the genes whose GO ID is owned by a gene of the association 

        #Compute for all associating gene and for each GO id associated with the gene the probability. 
        for name in asso :
            ann_name_gene = ann[ann["symbol"] == name]
            id = ann_name_gene["id"]
            tmp = []
            for i in (list(id.unique())) : 
                gene_imply = ann[ann['id']==i]
                K_hypergeom = gene_imply.shape[0] # get be the number of genes in the genome with annotation
                curr_regdom = regdom.loc[regdom["Name"].isin(list(gene_imply["symbol"]))]
                k_hypergeom = curr_regdom.loc[curr_regdom["Name"].isin(asso)].shape[0] # get the number of genes in the test gene set with annotation                
                tmp.append((i,gene_imply.iloc[0]["name"],K_hypergeom,k_hypergeom)) 
            res.update({elem[0]:[ elem[1], hypergeom_cdf(hypergeom_total_number_gene,elem[2],hypergeom_gene_set,elem[3]),(elem[3]*hypergeom_total_number_gene)/(hypergeom_gene_set*elem[2]) ] for elem in tmp}) 
        return pd.DataFrame(res).transpose().rename(columns = {0:"go_term",1:"hypergeom_p_value",2:"hypergeometric_fold_enrichment"}).replace(0,np.nan).dropna().sort_values(by = "hypergeom_p_value")


    def enrichment(test_file,regdom_file,chr_size_file, annotation_file, binom=True,hypergeom=True):
        """
        This function is a wrapper of the 3 private methods: 
        * GREAT.__enrichment_binom_and_hypergeom 
        * GREAT.__enrichment_binom 
        * GREAT.__enrichment_hypergeom

        Parameters
        ----------
        test_file : pd.DataFrame
            Genomic set of peaks to be tested
        regdom_file : pd.DataFrame 
            Regulatory domain of all genes in the genome 
        chr_size_file :  pd.DataFrame
            Table with the size of each chromosome
        annotation_file : pd.DataFrame
            Table with the annotation of each gene in the genome
        binom : bool
            If True, the binomial test is used.
        hypergeom : bool
            If True, the hypergeometric test is used.

        Returns
        -------
        pd.DataFrame
            dataframe contains for every GO ID associate with a every associated gene the p-value for the hypergeometric test
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv")
        >>> enrichment = GREAT.enrichment(
        ...    test = test,
        ...    regdom = regdom,
        ...    chr_size_file = size,
        ...    ann = ann,
        ...    binom=True,
        ...    hypergeom=True
        ...    )
        >>> enrichment.head()
            |            | go_term                                                          |   binom_p_value |   hypergeom_p_value |
            |:-----------|:-----------------------------------------------------------------|----------------:|--------------------:|
            | GO:0045887 | positive regulation of synaptic growth at neuromuscular junction |     5.17744e-13 |          0.0029275  |
            | GO:0044721 | protein import into peroxisome matrix, substrate release         |     4.83812e-10 |          0.0029275  |
            | GO:0036250 | peroxisome transport along microtubule                           |     4.83812e-10 |          0.0029275  |
            | GO:0016561 | protein import into peroxisome matrix, translocation             |     6.31131e-10 |          0.00584656 |
            | GO:0047485 | protein N-terminus binding                                       |     1.2945e-09  |          0.0050377  |

        >>> enrichment = GREAT.enrichment(
        ...    test = test,
        ...    regdom = regdom,
        ...    ann = ann,
        ...    asso = get_association(test,regdom),
        ...    binom=True,
        ...    hypergeom=False
        ...    )
        >>> enrichment.head()
            |            | go_term                                                          |   binom_p_value |
            |:-----------|:-----------------------------------------------------------------|----------------:|
            | GO:0045887 | positive regulation of synaptic growth at neuromuscular junction |     5.17744e-13 |
            | GO:0044721 | protein import into peroxisome matrix, substrate release         |     4.83812e-10 |
            | GO:0036250 | peroxisome transport along microtubule                           |     4.83812e-10 |
            | GO:0016561 | protein import into peroxisome matrix, translocation             |     6.31131e-10 |
            | GO:0047485 | protein N-terminus binding                                       |     1.2945e-09  |

        >>> enrichment = GREAT.enrichment(
        ...    test = test,
        ...    regdom = regdom,
        ...    ann = ann,
        ...    asso = get_association(test,regdom),
        ...    binom=False,
        ...    hypergeom=True
        ...    )
        >>> enrichment.head()

        """
        if not binom and not hypergeom : 
            return False
        
        test,regdom,size,ann = GREAT.loader(test_file,regdom_file,chr_size_file, annotation_file)
        asso = get_association(test,regdom)# get the name of the regulatory domain associated to each genomic region in the test set


        if binom and hypergeom : 
            return GREAT.__enrichment_binom_and_hypergeom(test,regdom,size,ann,asso)

        elif binom : 
            return GREAT.__enrichment_binom(test,regdom,size,ann,asso)

        else : 
              return GREAT.__enrichment_hypergeom(test,regdom,ann,asso)

    def set_bonferroni(self,alpha:float=0.05): 
        """
        This function create new columns in the dataframe with the Bonferroni correction

        Parameters
        ----------
        alpha : float
            alpha value for the Bonferroni correction
        
        Returns
        -------
        pd.DataFrame
            dataframe new columns with the Bonferroni correction for each p-value
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv")
        >>> enrichment = great.tl.GREAT.enrichment("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv",binom=True,hypergeom=True)
        >>> bonferroni = GREAT.set_bonferroni(enrichment,alpha=0.05)
        >>> bonferroni.head()
        ...    |            | go_term                                                          |   binom_p_value |   hypergeom_p_value |   binom_bonferroni |   hypergeom_bonferroni |
        ...    |:-----------|:-----------------------------------------------------------------|----------------:|--------------------:|-------------------:|-----------------------:|
        ...    | GO:0045887 | positive regulation of synaptic growth at neuromuscular junction |     5.17744e-13 |          0.0029275  |        3.0754e-10  |                      1 |
        ...    | GO:0044721 | protein import into peroxisome matrix, substrate release         |     4.83812e-10 |          0.0029275  |        2.87384e-07 |                      1 |
        ...    | GO:0036250 | peroxisome transport along microtubule                           |     4.83812e-10 |          0.0029275  |        2.87384e-07 |                      1 |
        ...    | GO:0016561 | protein import into peroxisome matrix, translocation             |     6.31131e-10 |          0.00584656 |        3.74892e-07 |                      1 |
        ...    | GO:0047485 | protein N-terminus binding                                       |     1.2945e-09  |          0.0050377  |        7.68931e-07 |                      1 |

        """
        for col in self.columns : 
            if col in ["binom_p_value","hypergeom_p_value"] : 
                col_split = col.split("_")
                self[f"{col_split[0]}_bonferroni"] = multipletests(self[col], alpha=alpha, method='bonferroni')[1]
        return self 

    def set_fdr(self,alpha:float=0.05) : 
        """
        This function create new columns in the dataframe with the fdr correction

        Parameters
        ----------
        alpha : float
            alpha value for the fdr correction
        
        Returns
        -------
        pd.DataFrame
            dataframe new columns with the fdr correction for each p-value
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv")
        >>> enrichment = great.tl.GREAT.enrichment("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv",binom=True,hypergeom=True)
        >>> fdr = GREAT.set_fdr(enrichment,alpha=0.05)
        >>> fdr.head()
        ...    |            | go_term                                                          |   binom_p_value |   hypergeom_p_value |   binom_fdr |   hypergeom_fdr |
        ...    |:-----------|:-----------------------------------------------------------------|----------------:|--------------------:|------------:|----------------:|
        ...    | GO:0045887 | positive regulation of synaptic growth at neuromuscular junction |     5.17744e-13 |          0.0029275  | 3.0754e-10  |       0.0913909 |
        ...    | GO:0044721 | protein import into peroxisome matrix, substrate release         |     4.83812e-10 |          0.0029275  | 9.3723e-08  |       0.0913909 |
        ...    | GO:0036250 | peroxisome transport along microtubule                           |     4.83812e-10 |          0.0029275  | 9.3723e-08  |       0.0913909 |
        ...    | GO:0016561 | protein import into peroxisome matrix, translocation             |     6.31131e-10 |          0.00584656 | 9.3723e-08  |       0.0913909 |
        ...    | GO:0047485 | protein N-terminus binding                                       |     1.2945e-09  |          0.0050377  | 1.53786e-07 |       0.0913909 |

        """
        for col in self.columns : 
            if col in ["binom_p_value","hypergeom_p_value"] : 
                col_split = col.split("_")
                # self[f"{col_split[0]}_fdr"] = fdrcorrection(self[col], alpha=alpha)[1]
                self[f"{col_split[0]}_fdr"] = multipletests(self[col], alpha=alpha, method='fdr_bh')[1]
        return self 

    def set_threshold(self,colname:str, alpha:int=0.05) : 
        """
        This function allows to delete rows according to the p-value of the column taken as argument. By default the alpha value is 0.05

        Parameters
        ----------
        alpha : float
            alpha value for the fdr correction
        
        Returns
        -------
        pd.DataFrame
            dataframe with the rows deleted according to the p-value threshold
            
        Exemples 
        --------
        >>> test,regdom,size,ann = GREAT.loader("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv")
        >>> enrichment = great.tl.GREAT.enrichment("../../data/human/test_genomic_region.bed", "../../data/human/regulatory_domain.bed", "../../data/human/chr_size.bed", "../../data/human/ontologies.csv",binom=True,hypergeom=True)
        >>> enrichment.shape[0]
        ...    594

        >>> significant = GREAT.set_threshold(enrichment,colname="binom_p_value",alpha=0.05)
        >>> significant.shape[0]
        ...    310

        """
        if colname in self.columns: 
            self = self.loc[self[colname]<=alpha]
        return self 

###################################################################
########################### unfinish ##############################
###################################################################

    def get_nb_asso_per_region(test,regdom) : 
        """
        Function allowing from a file of genomic regions from CHIPseq 
        and a file of genomic regulatory domains to determine number of peaks 
        associated with each gene in the regulatory domain. 

        Parameters
        ----------
        test : str
            path of the file with the tests pics => columns: ["Chr","Chr_Start","Chr_End"]
        
        regdom : str
            path of the file with the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

        Returns
        -------
        res : dict
            dict with the number of associated genes per genomic region : key = associated gene, value = number of peaks associated with the gene 
            
        Exemples 
        --------
        test = pd.DataFrame({
        ...    "Chr":["chr1"],
        ...    "Chr_Start":[1052028],
        ...    "Chr_End": [1052049]})

        regdom = pd.DataFrame({
        ...    "Chr":["chr1","chr1"],
        ...    "Chr_Start":[1034992,1079306],
        ...    "Chr_End": [1115089,1132016],
        ...    "Name":["RNF223","C1orf159"],
        ...    "tss":[1074306,1116089],
        ...    "strand":['-','-']})

        >>> get_association(test,regdom)        
            {'RNF223':2}
        
        """
        res = {}
        test,regdom,_,_ = GREAT.loader(test,regdom,None,None)
        for i in range(test.shape[0]) :
            currTest = test.iloc[i]
            regdom_curr_test = regdom.loc[regdom["Chr"] == currTest["Chr"]].sort_values("Chr_Start")
            regdom_inf = regdom_curr_test.loc[regdom_curr_test["tss"] <= currTest["Chr_Start"]]
            regdom_sup = regdom_curr_test.loc[regdom_curr_test["tss"] >= currTest["Chr_End"]]
            try : 
                if regdom_inf.iloc[-1]["Name"] not in res.keys() : 
                    res[regdom_inf.iloc[-1]["Name"]] = 1 
                else :
                    res[regdom_inf.iloc[-1]["Name"]] += 1
            except :
                pass

            try :
                if regdom_sup.iloc[-1]["Name"] not in res.keys() : 
                    res[regdom_sup.iloc[-1]["Name"]] = 1 
                else :
                    res[regdom_sup.iloc[-1]["Name"]] += 1
            except : 
                pass
        return res

    def get_dist_to_tss(test,regdom) : 
        """
        Function allowing from a file of genomic regions from CHIPseq 
        and a file of genomic regulatory domains to determine the distance from peaks 
        to the transcription start site of the associated gene

        Parameters
        ----------
        test : str
            path of the file with the tests pics => columns: ["Chr","Chr_Start","Chr_End"]
        
        regdom : str
            path of the file with the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

        Returns
        -------
        res : dict
            dict with the distance from tss to the associated genes : key = associated gene, value = distance from peaks to tss 
            
        Exemples 
        --------
        test = pd.DataFrame({
        ...    "Chr":["chr1"],
        ...    "Chr_Start":[1052028],
        ...    "Chr_End": [1052049]})

        regdom = pd.DataFrame({
        ...    "Chr":["chr1","chr1"],
        ...    "Chr_Start":[1034992,1079306],
        ...    "Chr_End": [1115089,1132016],
        ...    "Name":["RNF223","C1orf159"],
        ...    "tss":[1074306,1116089],
        ...    "strand":['-','-']})

        >>> get_association(test,regdom)        
            {'RNF223':[-22278]}
        
        """
        res = {}
        test,regdom,_,_ = GREAT.loader(test,regdom,None,None)
        for i in range(test.shape[0]) :
            currTest = test.iloc[i]
            regdom_curr_test = regdom.loc[regdom["Chr"] == currTest["Chr"]].sort_values("Chr_Start")
            regdom_inf = regdom_curr_test.loc[regdom_curr_test["tss"] <= currTest["Chr_Start"]]
            regdom_sup = regdom_curr_test.loc[regdom_curr_test["tss"] >= currTest["Chr_End"]]
            try : 
                if regdom_inf.iloc[-1]["Name"] not in res.keys() : 
                    res[regdom_inf.iloc[-1]["Name"]] = [currTest["Chr_Start"] - res[regdom_inf.iloc[-1]["tss"]]]
                else : 
                    res[regdom_inf.iloc[-1]["Name"]].append(currTest["Chr_Start"] - res[regdom_inf.iloc[-1]["tss"]])
            except :
                pass

            try :
                if regdom_sup.iloc[-1]["Name"] not in res.keys() : 
                    res[regdom_sup.iloc[-1]["Name"]] = [currTest["Chr_End"] - res[regdom_sup.iloc[-1]["tss"]]]
                else :
                    res[regdom_sup.iloc[-1]["Name"]].append(currTest["Chr_End"] - res[regdom_sup.iloc[-1]["tss"]])
            except : 
                pass
        return res



In [4]:
rgreat = importr('rGREAT')
ranges = importr('GenomicRanges')

In [5]:
test = "../../data/tests/test_data/input/01_random.bed"
regdom = "../../data/human/hg38/regulatory_domain.bed"
great_out = "../../data/tests/test_data/output/10_MAX_output_great_webserver.tsv"
great_asso = "../../data/tests/test_data/output/10_MAX_gene_asso.tsv"
size = "../../data/human/hg38/chr_size.bed"

In [12]:
df = r["read.csv"](test,sep='\t')
seqname = rpy2.robjects.StrVector(["seqnames", "seqname","chromosome", "X.Chr","chr", "chromosome_name","seqid"])
end = rpy2.robjects.StrVector(['end', 'stop'])
df = ranges.makeGRangesFromDataFrame(df,seqnames_field=seqname)#,keep_extra_columns=False,ignore_strand=False,seqinfo=None,start_field='start', end_field=end, strand_field='strand',starts_in_df_are_0based=False)

    # great calculation
local = rgreat.great(df, "msigdb:C5", f"GREAT:hg38",verbose=False)
local = rgreat.getEnrichmentTables(local)

time.sleep(30)

R[write to console]: * use genome 'hg38'.

R[write to console]: * TSS extension mode is 'basalPlusExt'.

R[write to console]: * construct the basal domains by extending 5000bp to upstream and 1000bp to downsteram of TSS.

R[write to console]: * calculate distances to neighbour regions.

R[write to console]: * extend to both sides until reaching the neighbour genes or to the maximal extension (1000000bp).



In [7]:
# greatpy calculation
greatpy = GREAT.enrichment(
    test_file=test,
    regdom_file=regdom,
    chr_size_file=size,
    annotation_file=f"../../data/human/ontologies.csv",
    binom=True,
    hypergeom=True,
    )

In [8]:
greatpy

,go_term,binom_p_value,binom_fold_enrichment,n_binom,k_binom,p_binom,hypergeom_p_value,hypergeometric_fold_enrichment,N_hyper,K_hyper,n_hyper,k_hyper
GO:0045887,positive regulation of synaptic growth at neur...,5.20389e-13,2.18004e+05,104,4,1.83483e-05,3.08889e-03,3.23741e+02,18777,1,58,1
GO:0044721,"protein import into peroxisome matrix, substra...",4.84198e-10,3.94065e+04,104,4,1.01506e-04,3.08889e-03,3.23741e+02,18777,1,58,1
GO:0036250,peroxisome transport along microtubule,4.84198e-10,3.94065e+04,104,4,1.01506e-04,3.08889e-03,3.23741e+02,18777,1,58,1
GO:0016561,"protein import into peroxisome matrix, translo...",6.31855e-10,3.68645e+04,104,4,1.08505e-04,6.16839e-03,1.61871e+02,18777,2,58,1
GO:0030576,Cajal body organization,1.49190e-09,2.97236e+04,104,4,1.34573e-04,1.83932e-02,5.39569e+01,18777,6,58,1
...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0006355,"regulation of transcription, DNA-templated",9.93956e-01,2.08617e+01,104,1,4.79347e-02,8.40449e-01,5.54352e-01,18777,584,58,1
GO:0006915,apoptotic process,9.95014e-01,2.01230e+01,104,1,4.96944e-02,5.39651e-01,1.11443e+00,18777,581,58,2
GO:0042803,protein homodimerization activity,9.98551e-01,2.44788e+01,104,2,8.17033e-02,9.54394e-01,3.33067e-01,18777,972,58,1
GO:0005737,cytoplasm,9.98903e-01,6.55886e+01,104,24,3.65917e-01,9.92060e-01,5.70469e-01,18777,5675,58,10


In [9]:
print(local.names)

 [1] "id"                    "genome_fraction"       "observed_region_hits" 
 [4] "fold_enrichment"       "p_value"               "p_adjust"             
 [7] "mean_tss_dist"         "observed_gene_hits"    "gene_set_size"        
[10] "fold_enrichment_hyper" "p_value_hyper"         "p_adjust_hyper"       



In [36]:
name_local = list(local.rx2("id")) 
name_local = [" ".join(cdc.lower().split("_")[1:]) for cdc in list(local.rx2("id"))]
local_df = pd.DataFrame({
    "name": name_local,
    "binom_p_val" : list(local.rx2("p_value")),
    "hyper_p_val" : list(local.rx2("p_value_hyper")),
    "p_binom" : list(local.rx2("genome_fraction")),
    "k_binom" : list(local.rx2("observed_region_hits"))
    })
local_df

,name,binom_p_val,hyper_p_val,p_binom,k_binom
0,retrograde vesicle mediated transport golgi to...,3.55271e-15,1.12145e-02,4.37301e-03,1.20000e+01
1,golgi vesicle transport,1.36486e-11,1.30739e-02,2.40640e-02,1.70000e+01
2,regulation of deoxyribonuclease activity,1.71753e-09,4.13466e-04,5.72306e-04,5.00000e+00
3,flemming body,2.95092e-08,5.20334e-03,3.67626e-03,7.00000e+00
4,cis golgi network,3.60733e-08,4.77428e-05,5.83826e-03,8.00000e+00
...,...,...,...,...,...
536,regulation of intracellular signal transduction,9.96128e-01,7.61276e-01,1.65203e-01,6.00000e+00
537,regulation of protein modification process,9.97033e-01,9.51139e-01,1.51710e-01,5.00000e+00
538,synapse,9.97216e-01,9.16143e-01,1.70161e-01,6.00000e+00
539,cellular response to stress,9.98265e-01,9.26567e-01,1.59363e-01,5.00000e+00


In [37]:
ann = pd.read_csv("../../data/human/ontologies.csv",sep=';',comment='#',header=0,)
ann["name"] = ann['name'].str.lower()

In [38]:
go = []
for i in range(local_df.shape[0]) : 
    curr = local_df.iloc[i]

    if ann.loc[ann["name"].isin([curr["name"]])].shape[0]>0 : 
        go.append(ann.loc[ann["name"].isin([curr["name"]])].iloc[0]["id"])
    else :
        go.append("")
local_df["id"] = go 
local_df = local_df.loc[local_df["id"]!=""]

In [39]:
loc_gpy = local_df[local_df["id"].isin(list(greatpy.index))]
gpy_loc = greatpy[greatpy.index.isin(list(local_df["id"]))]

In [40]:
gpy_loc[gpy_loc.index == "GO:0043547"]

,go_term,binom_p_value,binom_fold_enrichment,n_binom,k_binom,p_binom,hypergeom_p_value,hypergeometric_fold_enrichment,N_hyper,K_hyper,n_hyper,k_hyper
GO:0043547,positive regulation of GTPase activity,1.10493e-03,3.13301e+02,104,11,3.51100e-02,2.48282e-02,3.60715e+00,18777,359,58,4


In [41]:
loc_gpy[loc_gpy["id"] == "GO:0043547"]

,name,binom_p_val,hyper_p_val,p_binom,k_binom,id
113,positive regulation of gtpase activity,2.67021e-03,5.01449e-02,2.86761e-02,8.00000e+00,GO:0043547


In the online webserver we find the following results:

* binom_p_val : 6.6206e-3
* fold enr : 2.3810
* observed hit : 11
* coverage : 10.58%
* hyper_p_val : 3.6335e-2 
* fold enr : 3.1974
* obs gene hit : 4
* total gene set : 405
* hyper coverage : 6.90%
